In [1]:
import tenseal as ts
import utils

In [2]:
import os
from pathlib import Path
#Pour  utiliser la clée publique , sans savoir le nom du fichier qui la contient.
# Chemin vers le répertoire contenant la clé publique
keys_dir = "Keys_manipulator"

# Vérifier que le répertoire existe et contient exactement un fichier
if os.path.exists(keys_dir) and os.path.isdir(keys_dir):
    files = list(Path(keys_dir).glob('*'))
    
    if len(files) == 1:
        # Prendre le premier (et seul) fichier trouvé
        public_key_path = str(files[0])
        context = ts.context_from(utils.read_data(public_key_path))
    else:
        raise FileNotFoundError(f"Le répertoire {keys_dir} doit contenir exactement un fichier (contient {len(files)} fichiers)")
else:
    raise FileNotFoundError(f"Le répertoire {keys_dir} n'existe pas ou n'est pas un répertoire")


In [3]:
#Resultat covid
Parametre_a_manipuler_1 ="Data_received/SARS_Cov_2_exam_result_encrypted.txt"
resultat_covid=utils.read_data(Parametre_a_manipuler_1)
nombre_cas_positive= ts.ckks_vector_from(context,resultat_covid)
nombre_cas_positive =nombre_cas_positive.sum()
utils.write_data("outputs_manipulator/SARS_Cov_2_exam_result_SARS_Cov_2_exam_result_encrypted_computed.txt",nombre_cas_positive.serialize())



In [5]:
#score  fever, dry_cough, chest_pain
Parametre_a_manipuler_2_1 ="Data_received/chest_pain_encrypted.txt"
Parametre_a_manipuler_2_2 ="Data_received/dry_cough_encrypted.txt"
Parametre_a_manipuler_2_3 ="Data_received/fever_encrypted.txt"
chest_pain=utils.read_data(Parametre_a_manipuler_2_1)
dry_cough=utils.read_data(Parametre_a_manipuler_2_2)
fever=utils.read_data(Parametre_a_manipuler_2_3)
chest_pain= ts.ckks_vector_from(context,chest_pain)
dry_cough= ts.ckks_vector_from(context,dry_cough)
fever= ts.ckks_vector_from(context,fever)
score = chest_pain + dry_cough + fever
utils.write_data("outputs_manipulator/score_encrypted.txt",score.serialize())


In [ ]:
#Détéction de fièrvre dépassant 8 , sigmoid
# Approximation simple de seuil : f(x) = (x - 8) * k
# Si x < 5 → f(x) < 0 → proche de 0
# Si x > 5 → f(x) > 0 → proche de 1
# Parametre_a_manipuler_2_2 ="Data_received/fever_encrypted.txt"
